In [1]:
import os
from multiprocessing import Pool, Value
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from fractions import math

from skimage import color, exposure, io, img_as_ubyte
from skimage.transform import resize

from sklearn import preprocessing
from sklearn.externals import joblib

In [2]:
in_dir = "../input/images/"
out_dir = "../input/preprocessed-images/"
shape = 256
scaler_filename = "../models/images_StandardScaler.save"

In [3]:
def process_image(image_dir):
    image = io.imread(in_dir+image_dir)
    
    r = math.gcd(image.shape[0], image.shape[1])
    widht_ratio = int(image.shape[1] / r)
    height_ratio = int(image.shape[0] / r)

    # crop
    if widht_ratio > height_ratio*1.5:
        image = image[:,int((image.shape[1]-shape)/2):int((image.shape[1]-shape)/2+shape)]       
    elif height_ratio > widht_ratio*1.5:
        image = image[int((image.shape[0]-shape)/2):int((image.shape[0]-shape)/2+shape),:]

    image = resize(image, (shape, shape), mode='reflect', anti_aliasing=True)
    image = color.rgb2gray(image)
    image = exposure.equalize_hist(image)
    image = img_as_ubyte(image)
#    print("preprocessed: "+image_dir)
#    print("saved in: "+out_dir+image_dir)
    io.imsave(out_dir+image_dir,image)    
    return image

In [4]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

processes = 4

split_n = 100

scaler = preprocessing.MinMaxScaler(feature_range=(0,1))
# scaler = preprocessing.StandardScaler()

dir_list = np.array(os.listdir(in_dir))

pool = Pool(processes=processes)  # Num of CPUs

for sub_dir_list in np.array_split(dir_list, split_n):
    # crop, resize, rgb to grey and hist equalization.
    images = np.array(pool.map(process_image, sub_dir_list, chunksize = 8))

    # standarization or normalization
    images = np.reshape(images,(len(images),-1))
    scaler.partial_fit(images)
    print("scal")
    joblib.dump(scaler, scaler_filename)
    
joblib.dump(scaler, scaler_filename)

pool.close()
pool.terminate()

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


scal


Process ForkPoolWorker-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
Process ForkPoolWorker-3:
  File "<ipython-input-3-c12a7d39fb66>", line 14, in process_image
    image = resize(image, (shape, shape), mode='reflect', anti_aliasing=True)
  File "/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py", line 144, in resize
    cval=cval, mode=mode)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/filters.py", line 286, in gaussian_filter
    mode, cval, truncate)
  File "/usr/local/lib/python3.6/dist-packages/scipy/

KeyboardInterrupt: 

Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-3-c12a7d39fb66>", line 14, in process_image
    image = resize(image, (shape, shape), mode='reflect', anti_aliasing=True)
  File "/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py", line 144, in resize
    cval=cval, mode=mode)
  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/filters.py", line 286, in gaussian_filter
    mode, cval, truncate)
  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/filters.py", line 204, in gaussian_filter1d
    retur